In [1]:
import numpy as np
import pandas as pd

import utils
from sklearn.metrics.pairwise import paired_distances
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, LeaveOneOut, cross_val_predict
import matplotlib.pyplot as plt
import seaborn as sns 



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as pn

from sklearn.metrics.pairwise import cosine_similarity, paired_distances
from sklearn.preprocessing import StandardScaler, scale
from scipy.spatial import distance

from einops import rearrange, reduce, repeat


In [3]:
import importlib
importlib.reload(utils)


<module 'utils' from '/home/acercyc/Project/Keio Testing_analysis/src/utils.py'>

In [4]:
subjs = utils.ExpInfo.getSubjIDs()
task = utils.ExpInfo.taskName[0]


In [23]:
subj = subjs[0]
wSize = 60

# Group analysis

In [28]:
wSize = 60

def group(subj):
    df_beh = utils.LoadData.behaviorData(subj, task)
    _, h, _ = utils.LoadData.xhy(subj, task, wSize=wSize)
    _, h_disp, _ = utils.LoadData.xhy_disp(subj, task, wSize=wSize)
    dist_measure = 'euclidean'

    # dist_timeSeries = []
    # v_timeSeries = []
    d = []
    for iTrial in range(len(h)):
        h_trial = h[iTrial][:-1, :]
        h_disp_trial = h_disp[iTrial]

        # hidden action plan velocity
        vh = np.diff(h_trial, axis=0)
        vh_disp_dot = np.diff(h_disp_trial, axis=0)

        dist_timeSeries_ = paired_distances(h_trial, h_disp_trial, metric=dist_measure)
        v_timeSeries_ = paired_distances(vh, vh_disp_dot, metric=dist_measure)
        
        d.append({'dist_mean': dist_timeSeries_.mean(), 
                  'dist_sd': dist_timeSeries_.std(),
                  'v_mean': v_timeSeries_.mean(),
                  'v_sd': v_timeSeries_.std()})
        
        # change  distance to "similarity" index
        # d.append({'dist_mean': 1-dist_timeSeries_.mean(), 
        #         'dist_sd': 1-dist_timeSeries_.std(),
        #         'v_mean': 1-v_timeSeries_.mean(),
        #         'v_sd': 1-v_timeSeries_.std()})    
        
    df_ = pd.concat([df_beh, pd.DataFrame(d)], axis=1)
    X = df_[['dist_mean', 'v_mean']]
    # X = StandardScaler().fit_transform(X)

    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1'] = coef_[0][0]
    df_['b2'] = coef_[0][1]
    df_['b0'] = LR.intercept_[0]

    # standardise the data
    X = StandardScaler().fit_transform(X)
    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred_stdz'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1_stdz'] = coef_[0][0]
    df_['b2_stdz'] = coef_[0][1]
    df_['b0_stdz'] = LR.intercept_[0]
    
    # LeaveOneOut cross-validation
    cv = LeaveOneOut()
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced')
    df_['response_pred_cv'] = cross_val_predict(LR, X, y, cv=cv)

        
    # Only use action plan position
    # X = StandardScaler().fit_transform(X)
    X = df_[['dist_mean']]
    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred_p_only'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1_stdz_p_only'] = coef_[0][0]
    # df_['b2_stdz'] = coef_[0][1]
    df_['b0_stdz_p_only'] = LR.intercept_[0]
    
    # LeaveOneOut cross-validation
    cv = LeaveOneOut()
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced')
    df_['response_pred_p_only_cv'] = cross_val_predict(LR, X, y, cv=cv)
                
    return df_

df = utils.GroupOperation.map(group, subjs)
df = pd.concat(df, axis=0)


df.head()

Group loop |████████████████████████████████████████| 50/50 [100%] in 17.7s (2.80/s)                                    


,participant,trial no.,actual control,angular bias,response,group,dist_mean,dist_sd,v_mean,v_sd,...,b0,response_pred_stdz,b1_stdz,b2_stdz,b0_stdz,response_pred_cv,response_pred_p_only,b1_stdz_p_only,b0_stdz_p_only,response_pred_p_only_cv
0,K-Reg-H-10,1,0.30,0,0,H,0.637205,0.104790,0.167450,0.053875,...,1.976842,0,-2.694824,0.616395,-0.634225,0,0,-3.258059,1.941227,0
1,K-Reg-H-10,2,0.80,0,1,H,0.161334,0.040062,0.074640,0.066949,...,1.976842,1,-2.694824,0.616395,-0.634225,1,1,-3.258059,1.941227,1
2,K-Reg-H-10,3,0.30,0,0,H,0.677331,0.146095,0.194376,0.061812,...,1.976842,0,-2.694824,0.616395,-0.634225,0,0,-3.258059,1.941227,0
3,K-Reg-H-10,4,0.55,90,0,H,0.897086,0.210970,0.184965,0.064054,...,1.976842,0,-2.694824,0.616395,-0.634225,0,0,-3.258059,1.941227,0
4,K-Reg-H-10,5,0.30,90,0,H,0.949337,0.145878,0.229553,0.051487,...,1.976842,0,-2.694824,0.616395,-0.634225,0,0,-3.258059,1.941227,0


In [29]:
fn = utils.Save.savepath('ana_one_dot_predicting_individual_beh_profile_cv', 'prediction.csv')
df.to_csv(fn, index=False)

In [35]:
(df['response_pred_cv'] == df['response']).mean()

0.8396666666666667